In [2]:
import mikeio
import modelskill
import matplotlib.pyplot as plt 
import pandas as pd
from tqdm import tqdm
import numpy as np

In [22]:
Flinten7 = pd.read_csv("raw_data/smhi-opendata_2_3_35137_20241118_065734.csv",index_col=0,sep=";", parse_dates=True)
Flinten7.rename(columns = { 'Strömriktning':"Current direction", 'Kvalitet':"Quality_d",'Mätdjup (m)':"Measuring depth (m)_d",
                            'Strömhastighet':"Current speed", 'Kvalitet.1':"Quality_s",'Mätdjup (m).1':"Measuring depth (m)_s"},inplace=True)

# Filter data after time steps and quality flags.
Flinten7 = Flinten7.loc[(Flinten7.index >= "2014-01-01") & (Flinten7["Quality_d"] == "G") & (Flinten7["Quality_s"] == "G") ].drop(columns=["Quality_d","Quality_s","Unnamed: 7","Tidsutsnitt"])
Flinten7


,Current direction,Measuring depth (m)_d,Current speed,Measuring depth (m)_s
Datum Tid (UTC),,,,
2014-01-01 00:00:00,4.27,0,57.52,0.0
2014-01-01 00:00:00,32.56,2,47.79,2.0
2014-01-01 00:00:00,33.09,4,47.33,4.0
2014-01-01 00:00:00,36.80,6,49.19,6.0
2014-01-01 01:00:00,8.65,0,61.63,0.0
...,...,...,...,...
2024-11-17 22:00:00,224.35,6,32.92,6.0
2024-11-17 23:00:00,150.49,0,7.72,0.0
2024-11-17 23:00:00,197.60,2,35.49,2.0


In [16]:
# Convert Current speed and directions to u and v components

In [ ]:
# Calculate depth averaged u and v components